In [1]:
from rdflib import Graph, ConjunctiveGraph, RDF, RDFS, OWL
from SPARQLWrapper import SPARQLWrapper2
import pandas as pds
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tabulate import tabulate
import functools

## define function for printing pandas dataframes as markdown

In [2]:
def print_pandas_md(df):
    print(tabulate(df, tablefmt="pipe", headers="keys"))

## define function for converting sparql results to a dataframe

In [3]:
def sparql_to_df(results):
    data = {}
    for variable in results.variables:
        data[variable] = [x.value for x in results.getValues(variable)]
    return pds.DataFrame(data)

## define function to round values in an interval object

In [4]:
def round_interval(interval):
    left_val = round(interval.left)
    right_val = round(interval.right)
    return pds.Interval(left=left_val, right=right_val, closed='left')

## set connection to local triple store

In [5]:
sparql = SPARQLWrapper2("http://192.168.1.243:7200/repositories/EDR_NO_ES")

## define some standard prefixes to use in queries

In [6]:
prefixes = """
PREFIX obo: <http://purl.obolibrary.org/obo/>
PREFIX part_of: <http://purl.obolibrary.org/obo/BFO_0000050>
PREFIX has_part: <http://purl.obolibrary.org/obo/BFO_0000051>
PREFIX material: <http://purl.obolibrary.org/obo/OHD_0000000>
PREFIX tooth: <http://purl.obolibrary.org/obo/FMA_12516>
PREFIX restored_tooh: <http://purl.obolibrary.org/obo/OHD_0000189>
PREFIX surface: <http://purl.obolibrary.org/obo/FMA_no_fmaid_Surface_enamel_of_tooth>
PREFIX restored_surface: <http://purl.obolibrary.org/obo/OHD_0000208>
PREFIX restored_buccal: <http://purl.obolibrary.org/obo/OHD_0000222>
PREFIX restored_distal: <http://purl.obolibrary.org/obo/OHD_0000223>
PREFIX restored_mesial: <http://purl.obolibrary.org/obo/OHD_0000227>
PREFIX restored_labial: <http://purl.obolibrary.org/obo/OHD_0000225>
PREFIX restored_lingual: <http://purl.obolibrary.org/obo/OHD_0000226>
PREFIX restored_incisal: <http://purl.obolibrary.org/obo/OHD_0000224>
PREFIX restored_occlusal: <http://purl.obolibrary.org/obo/OHD_0000228>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ada_num: <http://purl.obolibrary.org/obo/OHD_0000065>
PREFIX patient: <http://purl.obolibrary.org/obo/OHD_0000012>
PREFIX male_patient: <http://purl.obolibrary.org/obo/OHD_0000054>
PREFIX female_patient: <http://purl.obolibrary.org/obo/OHD_0000049>
PREFIX provider: <http://purl.obolibrary.org/obo/OHD_0000051>
PREFIX procedure: <http://purl.obolibrary.org/obo/OHD_0000002>
PREFIX restoration_procedure: <http://purl.obolibrary.org/obo/OHD_0000004>
PREFIX participates_in: <http://purl.obolibrary.org/obo/BFO_0000056>
PREFIX has_participant: <http://purl.obolibrary.org/obo/BFO_0000057>
PREFIX has_specified_input: <http://purl.obolibrary.org/obo/OBI_0000293>
PREFIX has_specified_output: <http://purl.obolibrary.org/obo/OBI_0000299>
PREFIX sesame: <http://www.openrdf.org/schema/sesame#>
PREFIX birth_date: <http://purl.obolibrary.org/obo/OHD_0000050>
PREFIX npbrn_id: <http://purl.obolibrary.org/obo/OHD_0000273>
PREFIX occur_date: <http://purl.obolibrary.org/obo/OHD_0000015>
PREFIX cdt_code: <http://purl.obolibrary.org/obo/CDT_1000001>
PREFIX is_about: <http://purl.obolibrary.org/obo/IAO_0000136>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
"""

## find teeth that underwenta a procedure

In [9]:
q = """
%s
select 
    ?patient_id
    ?tooth_id
    ?tooth_num
    ?event
    ?event_date
    ?cdt_code
    ?cdt_label
    ?m
    ?o
    ?d
    ?b
    ?f
    ?l
    ?i
where {
    ?patient_i a patient: .
    ?tooth_i a tooth:;
             part_of: ?patient_i;
             sesame:directType ?tooth_t .
    ?tooth_t ada_num: ?ada_num .
            
    ?proc_i a procedure:;
            sesame:directType ?proc_t;
            has_specified_input: ?tooth_i;
            occur_date: ?occur_date .
    ?proc_t rdfs:label ?proc_label .
    
    ?code_i a cdt_code:;
               	sesame:directType ?code_t;
                is_about: ?proc_i .
    
    ?code_t dc:identifier ?cdt_code;
            obo:cdt_label ?cdt_label .
    
    optional {
        ?buccal_i a restored_buccal: .
        ?proc_i has_specified_output: ?buccal_i .
    }
    bind(if(bound(?buccal_i), 1, 0) as ?b)
    
    optional {
        ?distal_i a restored_distal: .
        ?proc_i has_specified_output: ?distal_i .
    }
    bind(if(bound(?distal_i), 1, 0) as ?d)
    
    optional {
        ?facial_i a restored_labial: .
        ?proc_i has_specified_output: ?facial_i .
    }
     bind(if(bound(?facial_i), 1, 0) as ?f)
    
    optional {
        ?incisal_i a restored_incisal: .
        ?proc_i has_specified_output: ?incisal_i .
    }
     bind(if(bound(?incisal_i), 1, 0) as ?i)
    
    optional {
        ?lingual_i a restored_lingual: .
        ?proc_i has_specified_output: ?lingual_i .
    }
     bind(if(bound(?lingual_i), 1, 0) as ?l)
    
    optional {
        ?mesial_i a restored_mesial: .
        ?proc_i has_specified_output: ?mesial_i .
    }
     bind(if(bound(?mesial_i), 1, 0) as ?m)
    
    optional {
        ?occlusal_i a restored_occlusal: .
        ?proc_i has_specified_output: ?occlusal_i .
    }
    bind(if(bound(?occlusal_i), 1, 0) as ?o)
    
    filter (!isblank(?proc_i))
    filter (!isblank(?tooth_i))
    filter (!isblank(?proc_i))
    
    bind(strafter(str(?patient_i), "patient/") as ?patient_id)
    bind(strafter(str(?tooth_i), "tooth/") as ?tooth_id)
    bind(str(?proc_label) as ?event) # removes the laguage tags
    bind(strafter(?ada_num, "Tooth ") as ?tooth_num)
    bind(strbefore(str(?occur_date), "T") as ?event_date)
    
}
""" % prefixes
# print(q)

In [ ]:
sparql.setQuery(q)
res = sparql.query()
tooth_proc_df = sparql_to_df(res)

In [ ]:
tooth_proc_df.drop_duplicates(inplace=True) # make sure there are not duplicates
len(tooth_proc_df)

In [ ]:
patient_df.to_csv('triplestore-tooth-procedure-info.tsv', sep='\t')